In [3]:
import numpy as np
from pyscf import gto, scf, ao2mo, cc, fci

# --- Parameters ---
n_rows = 4
n_cols = 4
Ns = n_rows * n_cols  # 6 sites
Ne = 16  # 6 electrons (half-filling)
U = 4.0
t = 1.0

print(f"Lattice: {n_rows}x{n_cols} ({Ns} sites)")
print(f"Electrons: {Ne}")
print(f"U = {U}, t = {t}")

# --- Construct Hamiltonian Integrals ---

# 1-electron (hopping)
h1 = np.zeros((Ns, Ns))
for r in range(n_rows):
    for c in range(n_cols):
        i = r * n_cols + c
        
        # Right neighbor
        if c + 1 < n_cols:
            j = r * n_cols + (c + 1)
            h1[i, j] = h1[j, i] = -t
            
        # Down neighbor
        if r + 1 < n_rows:
            j = (r + 1) * n_cols + c
            h1[i, j] = h1[j, i] = -t

# 2-electron (interaction U)
# PySCF uses chemist notation (ij|kl) = \int i* j k* l
# Hubbard is U sum_i n_{i,up} n_{i,down}
# This corresponds to spatial integral (ii|ii) = U
eri = np.zeros((Ns, Ns, Ns, Ns))
for i in range(Ns):
    eri[i, i, i, i] = U

# --- Setup PySCF Molecule ---
# Create 6 Hydrogen atoms far apart to get 6 1s orbitals (orthogonal)
# Distance 100.0 ensures negligible overlap by default (though we overwrite it).
atom_list = []
for i in range(Ns):
    atom_list.append(f"H {i*10.0} 0 0")

mol = gto.M(
    atom=atom_list,
    basis='sto-3g',
    unit='Bohr',
    verbose=3 # useful for debugging
)
mol.nelectron = Ne
mol.spin = 0 # 3 up, 3 down
mol.incore_anyway = True # Ensure integrals are used in-core
# Remove nuclear repulsion
Enuc = mol.energy_nuc()
print(f"Nuclear Repulsion Energy (removed): {Enuc:.8f}")
mol.energy_nuc = lambda *args: 0.0

# --- Mean-Field (UHF) ---
# We use UHF to allow symmetry breaking (e.g. AFM) if needed,
# though standard init guess might land on RHF-like solution.
mf = scf.UHF(mol)

# Override matrix elements
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(Ns)
mf._eri = ao2mo.restore(8, eri, Ns) # Use 8-fold symmetry for real integrals
# Or just keep it dense if small enough (6 sites is tiny)
# Let's use ao2mo.restore(1, eri, Ns) for no symmetry to be safe first.
mf._eri = ao2mo.restore(1, eri, Ns)

# Initial guess
# We can provide a custom density matrix to break spin symmetry (AFM)
# Or let it converge to whatever it finds.
# Let's try to break symmetry slightly.
dm_alpha = np.zeros((Ns, Ns))
dm_beta = np.zeros((Ns, Ns))
for i in range(Ns):
    if i % 2 == 0:
        dm_alpha[i, i] = 1.0 # Up
    else:
        dm_beta[i, i] = 1.0 # Down
# If Ne=6, this puts 3 up, 3 down. Perfect.
dm_init = np.array([dm_alpha, dm_beta])

print("Running UHF...")
mf.kernel(dm0=dm_init)
# mf.kernel()

print(f"UHF Energy: {mf.e_tot:.8f}")

# --- CCSD ---
print("Running CCSD...")
mycc = cc.CCSD(mf)
mycc.kernel()
print(f"CCSD Energy: {mycc.e_tot:.8f}")

# --- UCCSD ---
# pyscf.cc.CCSD automatically dispatches to UCCSD if mf is UHF.
# But we can explicitly check object type.
print(f"CC Object Type: {type(mycc)}")

# Solving lambda equations to get density matrices if needed (for properties)
# mycc.solve_lambda()

# Comparison with FCI (Exact Diagonalization)
print("Running FCI for benchmark...")
# Create a fake object for FCI since it needs h1 and eri
# pyscf.fci.FCI needs a mean-field object or explicit integrals.
# direct_spin1 is for RHF, direct_uhf is for UHF.
# We can use solver generic.

# fs = fci.FCI(mf)
# fs.kernel()
# print(f"FCI Energy: {fs.e_tot:.8f}")

# diff = mycc.e_tot - fs.e_tot
# print(f"CCSD - FCI: {diff:.8f}")


Lattice: 4x4 (16 sites)
Electrons: 16
U = 4.0, t = 1.0
Nuclear Repulsion Energy (removed): 3.80916639
Running UHF...
converged SCF energy = -5.73728501752772  <S^2> = 5.4923123  2S+1 = 4.7926245
UHF Energy: -5.73728502
Running CCSD...
UCCSD not converged
E(UCCSD) = -6.114527226724138  E_corr = -0.3772422091964198
CCSD Energy: -6.11452723
CC Object Type: <class 'pyscf.cc.uccsd.UCCSD'>
Running FCI for benchmark...
